# boardcnn.py EXPLAINED

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf

In [ ]:
from cornersdetect import cornersdetection
from chessutils import find_coeffs

In [ ]:
colors = np.array([' ', 'w', 'b'])
figures = np.array([' ','p','b','n','r','q','k'])

In [ ]:
NUMCELL = 8
CELL = 60
IMGSIZE = NUMCELL*CELL

In [ ]:
#Loading models
model = tf.keras.models.load_model('models/unet_board_v4.h5')
classifier = tf.keras.models.load_model('models/class_figure_col_v4.h5', compile=False)

In [ ]:
#Folder with images
#PATH_TO_TST = 'test'
PATH_TO_TST = '../../out/'

In [ ]:
#Images in folder
fl = [os.path.join(PATH_TO_TST, f) for f in os.listdir(PATH_TO_TST) if f.split('.')[-1] == 'png']
fl

In [ ]:
filename = fl[0]

In [ ]:
#Loading image
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (IMGSIZE,IMGSIZE))
feed = np.expand_dims(img/255, axis=(0,-1))
feed.shape

In [ ]:
#Detecting characteristic points
pred = np.squeeze(model.predict(feed))
points = np.argmax(pred, axis=-1)

In [ ]:
#Visualizing
fig, axx = plt.subplots(nrows=1, ncols=2, figsize=(10,5), sharey=True)
axx[0].imshow(img, cmap='gray')
axx[1].imshow(points, cmap='gray')
fig.tight_layout()
plt.show()

In [ ]:
corners = cornersdetection(points)
corners

In [ ]:
xyc = np.mean(corners, axis=0)
xyc

In [ ]:
half = 0.5*np.max([np.sum(np.abs(p-xyc)) for p in corners])
half

In [ ]:
newc = xyc + half*np.array([[-1,-1],[1,-1],[-1,1],[1,1]])
newc

In [ ]:
coeffs = find_coeffs(newc, corners)

In [ ]:
brdsize = half / 6 * NUMCELL

In [ ]:
#Transforming/cropping
img = Image.fromarray(img)
img = img.transform(img.size, Image.PERSPECTIVE, coeffs, Image.BICUBIC, fill='white')
img = img.crop((xyc[0]-brdsize, xyc[1]-brdsize, xyc[0]+brdsize, xyc[1]+brdsize))

In [ ]:
img